In [1]:
!pip install transformers kaggle -q

     |████████████████████████████████| 778kB 3.1MB/s 
     |████████████████████████████████| 3.0MB 32.9MB/s 
     |████████████████████████████████| 890kB 44.3MB/s 
     |████████████████████████████████| 1.1MB 46.2MB/s 


In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets list

ref                                                               title                                             size  lastUpdated          downloadCount  
----------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  
gpreda/covid19-tweets                                             COVID19 Tweets                                    18MB  2020-08-12 10:07:39           1489  
christianlillelund/passenger-list-for-the-estonia-ferry-disaster  The Estonia Disaster Passenger List               14KB  2020-07-26 15:40:17            401  
futurecorporation/epitope-prediction                              COVID-19/SARS B-cell Epitope Prediction            1MB  2020-07-24 02:53:28            351  
Cornell-University/arxiv                                          arXiv Dataset                                      2GB  2020-05-06 23:18:35           1215  
ahsen1330/us-police-shootings                 

In [6]:
!kaggle datasets download -d parthplc/facebook-hateful-meme-dataset

100% 3.35G/3.35G [00:39<00:00, 50.2MB/s]
100% 3.35G/3.35G [00:39<00:00, 90.2MB/s]


In [7]:
!unzip facebook-hateful-meme-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/img/49762.png      
  inflating: data/img/49785.png      
  inflating: data/img/49786.png      
  inflating: data/img/49802.png      
  inflating: data/img/49805.png      
  inflating: data/img/49806.png      
  inflating: data/img/49807.png      
  inflating: data/img/49810.png      
  inflating: data/img/49826.png      
  inflating: data/img/49831.png      
  inflating: data/img/49832.png      
  inflating: data/img/49836.png      
  inflating: data/img/49850.png      
  inflating: data/img/49856.png      
  inflating: data/img/49861.png      
  inflating: data/img/49863.png      
  inflating: data/img/49867.png      
  inflating: data/img/49870.png      
  inflating: data/img/50124.png      
  inflating: data/img/50126.png      
  inflating: data/img/50129.png      
  inflating: data/img/50137.png      
  inflating: data/img/50142.png      
  inflating: data/img/50146.png      
  inflating: data/img/50148.png      

In [ ]:
import transformers

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from cloud_tpu_client import Client

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  print("Invalid device or cannot modify virtual devices once initialized.")
  pass

In [ ]:
def preprocess(string):
    string = string.lower()
    return re.sub(r"[\W_]+", " ", string)

In [ ]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]
train_df["text"] = train_df["text"].apply(preprocess)

In [ ]:
val_df = pd.read_json('data/dev.jsonl', lines = True)
val_df["img"] = "data/" + val_df["img"]
val_df["text"] = val_df["text"].apply(preprocess)

In [ ]:
img_width = 128
img_height = 128

In [ ]:
embeds = pickle.load(open("data/embeds.pickle", "rb"))
val_embeds = pickle.load(open("data/val-embeds.pickle", "rb"))

In [ ]:
def build_model():
    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    embeds = tf.keras.layers.Input(name = "embeds", shape = (768,))

    extractor = tf.keras.applications.EfficientNetB0(include_top = False, \
                                                 input_tensor = input_img, weights = "imagenet")
    
    extractor.trainable = False
    
    for layer in extractor.layers:
        if layer.name.startswith("block7"):
            layer.trainable = True

    x = tf.keras.layers.GlobalMaxPooling2D()(extractor.output)
    
    x = tf.keras.layers.concatenate([x, embeds])
    
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = tf.keras.models.Model([input_img, embeds], out)

    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    return model

In [ ]:
model = build_model()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 128, 128, 3)  0           image[0][0]                      
__________________________________________________________________________________________________
normalization (Normalization)   (None, 128, 128, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 129, 129, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [ ]:
def encode_single_sample(img_path, label, embeds):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, img_width])
    return {"image": img, "label": label, "embeds": embeds}

In [ ]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, embeds):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.embeds = embeds
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_embeds = self.embeds[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        y = np.zeros((self.batch_size,) + (768,), dtype = "float32")
        z = np.zeros((self.batch_size,), dtype = "uint8")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_embeds):
            sample = encode_single_sample(j,k,l)
            x[i] = sample["image"].numpy().tolist()
            y[i] = sample["embeds"]
            z[i] = sample["label"]
        
        return [x,y], z

In [ ]:
train_gen = HatefulMemes(8, train_df["img"].values.tolist()[:7500], train_df["label"].values.tolist()[:7500], embeds[:7500])
val_gen = HatefulMemes(8, train_df["img"].values.tolist()[7500:], train_df["label"].values.tolist()[7500:], embeds[7500:])

In [ ]:
epochs = 100
early_stopping_patience = 5

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_accuracy", patience = early_stopping_patience, restore_best_weights = True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'hateful.h5', monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max'
)

# Train the model
history = model.fit(
    train_gen,
    validation_data = val_gen, 
    epochs = epochs,
    callbacks = [early_stopping, checkpoint]
)

Epoch 1/100
937/937 [==============================] - ETA: 0s - loss: 0.8524 - accuracy: 0.6306
Epoch 00001: val_accuracy improved from -inf to 0.68300, saving model to hateful.h5
937/937 [==============================] - 312s 332ms/step - loss: 0.8524 - accuracy: 0.6306 - val_loss: 0.6098 - val_accuracy: 0.6830
Epoch 2/100
937/937 [==============================] - ETA: 0s - loss: 0.6101 - accuracy: 0.6789
Epoch 00002: val_accuracy improved from 0.68300 to 0.70200, saving model to hateful.h5
937/937 [==============================] - 294s 314ms/step - loss: 0.6101 - accuracy: 0.6789 - val_loss: 0.5774 - val_accuracy: 0.7020
Epoch 3/100
937/937 [==============================] - ETA: 0s - loss: 0.5916 - accuracy: 0.6938
Epoch 00003: val_accuracy did not improve from 0.70200
937/937 [==============================] - 282s 301ms/step - loss: 0.5916 - accuracy: 0.6938 - val_loss: 0.6045 - val_accuracy: 0.6690
Epoch 4/100
937/937 [==============================] - ETA: 0s - loss: 0.5835 